In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords # Import the stop word list
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import random
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, confusion_matrix,accuracy_score

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
test_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [5]:
train_data['neutral'] = train_data.apply(lambda x: 0 if sum(x[2:8])>=1 else 1 ,axis = 1)
print(train_data.head())

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  neutral  
0             0        0       0       0              0        1  
1             0        0       0       0              0        1  
2             0        0       0       0              0        1  
3             0        0       0       0              0        1  
4             0        0       0       0              0        1  


In [6]:
def process_comment_text(txt):
    ntxt = re.sub(r"[^a-zA-Z]", " ", txt)
    ntxt = ntxt.lower()
    return ntxt

train_data['processed'] = train_data.comment_text.apply(process_comment_text)
test_data['processed'] = test_data.comment_text.apply(process_comment_text)
print(train_data.head())
print(test_data.head())

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  neutral  \
0             0        0       0       0              0        1   
1             0        0       0       0              0        1   
2             0        0       0       0              0        1   
3             0        0       0       0              0        1   
4             0        0       0       0              0        1   

                                           processed  
0  explanation why the edits made under my userna

# Tfidf Vectorizer and Regression - 1VsAll

In [7]:
vect = TfidfVectorizer(min_df=3,stop_words='english').fit(train_data.processed.values)
x_train_vectorized = vect.transform(train_data.processed.values)
x_test_vectorized = vect.transform(test_data.processed.values)

In [8]:
def class_model(data,labels):
    negative_ind = np.where(labels == 0)[0]
    positive_ind = np.where (labels == 1)[0]
    balance_negative = np.random.choice(negative_ind,size = len(positive_ind),replace = False)
    train_ind = np.concatenate((positive_ind,balance_negative))
    random.shuffle(train_ind)
    data = data[train_ind,:]
    labels = labels[train_ind]
    
    model = LogisticRegressionCV(
        Cs=list(np.power(10.0, np.arange(-10, 10)))
        ,penalty='l2'
        ,scoring='roc_auc'
        ,cv=5
        ,random_state=777
        ,max_iter=10000
        ,fit_intercept=True
        ,solver='newton-cg'
        ,tol=10
        ,refit = True
    )
    model.fit(data, labels)
    return model

In [9]:
columns = ['id']
index = test_data.index # array of numbers for the number of samples
res = pd.DataFrame(columns=columns, index = index)
res['id'] = test_data['id']

for c in train_data.columns[2:8]:
    labels = np.array(train_data[c])
    mm = class_model(x_train_vectorized,labels)
    predictions = mm.predict_proba(x_test_vectorized)
    res[c] = predictions[:,np.squeeze(np.where(mm.classes_==1))]

In [10]:
print(res)

                      id     toxic  severe_toxic   obscene    threat  \
0       00001cee341fdb12  0.999961      0.841339  0.999793  0.676522   
1       0000247867823ef7  0.009910      0.216405  0.008653  0.313826   
2       00013b17ad220c46  0.119279      0.197417  0.089003  0.326146   
3       00017563c3f7919a  0.007098      0.114330  0.004768  0.258136   
4       00017695ad8997eb  0.133988      0.153090  0.070369  0.278133   
5       0001ea8717f6de06  0.046924      0.078524  0.023245  0.252674   
6       00024115d4cbde0f  0.011704      0.090022  0.009392  0.242788   
7       000247e83dcc1211  0.936562      0.273257  0.491003  0.406894   
8       00025358d4737918  0.190578      0.150489  0.038951  0.337208   
9       00026d1092fe71cc  0.031711      0.123171  0.012189  0.272726   
10      0002eadc3b301559  0.698406      0.088031  0.960841  0.264753   
11      0002f87b16116a7f  0.107049      0.164873  0.220782  0.331453   
12      0003806b11932181  0.054003      0.093678  0.018100  0.24

In [11]:
res.to_csv('tfid_reg.csv',index=False)